In [ ]:
from pyspark import SparkContext as sc
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession, functions, types
from pyspark.sql.types import *
from IPython.display import display
import pandas as pd

In [ ]:
sc = sc(appName="attraction")
sqlContext = SQLContext(sc)
spark = SparkSession.builder.appName('attraction dataset combiner').getOrCreate()

In [ ]:
det1 = 'outputs/attractions_details_batch1.json'
det2 = 'outputs/attractions_details_batch2.json'
rev1 = 'outputs/attractions_reviews_batch1.json'
rev2 = 'outputs/attractions_reviews_batch2.json'
attraction_cat = 'outputs/attractions_cat.json'

In [ ]:
cat_df = pd.read_json('outputs/attractions_cat.json',orient='records')
cat_df['attraction_id'] = cat_df.index
cat_df = cat_df.rename(index=str,columns={"attraction": "url"})

In [ ]:
schema = StructType([ StructField("url", StringType(), True),
                      StructField("category", StringType(), True),
                      StructField("attraction_id", IntegerType(), True)])
cat_df_spark = spark.createDataFrame(cat_df, schema)

In [ ]:
cat_df_spark.show()

In [ ]:
det1_df = spark.read.json(det1)
det2_df = spark.read.json(det2)

In [ ]:
det_joined = det1_df.union(det2_df)
det_joined = det_joined.join(cat_df_spark.select(['category','attraction_id']),"attraction_id")

if det1_df.count() + det2_df.count() == det_joined.count():
    print("combined successfully!")
else:
    print("error in dataframe union!")

In [ ]:
det_joined.write.parquet('outputs/attraction_details',mode='overwrite')

In [ ]:
rev1_df = spark.read.json(rev1)
rev2_df = spark.read.json(rev2)

In [ ]:
rev_joined = rev1_df.union(rev2_df)

if rev1_df.count() + rev2_df.count() == rev_joined.count():
    print("combined successfully!")
else:
    print("error in dataframe union!")

In [ ]:
rev_joined.write.parquet('outputs/attraction_reviews',mode='overwrite')

In [ ]:
print(det_joined.count())
det_joined.show()

In [ ]:
print(rev_joined.count())
rev_joined.show()